In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import numpy as np
import plotly.express as px
import folium
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from folium import plugins
from sklearn.preprocessing import StandardScaler
from folium.plugins import HeatMap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [32]:
total_data = pd.read_csv('Taiwan_with_both.csv')
total_data = total_data[total_data["Total price (10,000 yuan)"]<3000]
total_data=total_data.reset_index().drop(columns="index")

#total_data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
print(len(total_data))
# split the DataFrame into 3 different subsets
trainmask = (total_data['transaction date'] < "110/01/01")
#validatemask = (total_data['transaction date'] >= "110/01/01") & (total_data['transaction date'] <= "110/03/31")
testmask = (total_data['transaction date'] >= "110/01/01")
train = total_data.loc[trainmask].copy()
test = total_data.loc[testmask].copy()
#validate = total_data.loc[validatemask].copy()

368349


In [33]:
train[train["hospitals_nearby_0_place_id"].isna()]

,Unnamed: 0,Lot location or house number,transaction date,"Total price (10,000 yuan)","Unit price (10,000 yuan/square meter)",Total area(ping),Proportion of main building,House age,management organization,elevator,City,geometry_lat,geometry_lng,Land,Construction,Car,Bedrooms,Living rooms,Bathrooms,Encoded_type,Encoded_floor,Encoded_purpose,Encoded_city,len_parks,parks_nearby_0_geometry_lat,parks_nearby_0_geometry_lng,parks_nearby_0_rating,parks_nearby_0_place_id,parks_nearby_0_user_ratings_total,parks_nearby_1_geometry_lat,parks_nearby_1_geometry_lng,parks_nearby_1_rating,parks_nearby_1_place_id,parks_nearby_1_user_ratings_total,parks_nearby_2_geometry_lat,parks_nearby_2_geometry_lng,parks_nearby_2_rating,parks_nearby_2_place_id,parks_nearby_2_user_ratings_total,parks_nearby_3_geometry_lat,parks_nearby_3_geometry_lng,parks_nearby_3_rating,parks_nearby_3_place_id,parks_nearby_3_user_ratings_total,parks_nearby_4_geometry_lat,parks_nearby_4_geometry_lng,parks_nearby_4_rating,parks_nearby_4_place_id,parks_nearby_4_user_ratings_total,len_hospitals,hospitals_nearby_0_geometry_lat,hospitals_nearby_0_geometry_lng,hospitals_nearby_0_rating,hospitals_nearby_0_place_id,hospitals_nearby_0_user_ratings_total,hospitals_nearby_1_geometry_lat,hospitals_nearby_1_geometry_lng,hospitals_nearby_1_rating,hospitals_nearby_1_place_id,hospitals_nearby_1_user_ratings_total,hospitals_nearby_2_geometry_lat,hospitals_nearby_2_geometry_lng,hospitals_nearby_2_rating,hospitals_nearby_2_place_id,hospitals_nearby_2_user_ratings_total,hospitals_nearby_3_geometry_lat,hospitals_nearby_3_geometry_lng,hospitals_nearby_3_rating,hospitals_nearby_3_place_id,hospitals_nearby_3_user_ratings_total,hospitals_nearby_4_geometry_lat,hospitals_nearby_4_geometry_lng,hospitals_nearby_4_rating,hospitals_nearby_4_place_id,hospitals_nearby_4_user_ratings_total
6,6,南區五權南一路５３號二樓之六,104/07/01,668.0,17.2,38.76,65.63%,3.0,1,NaN,Taichung,24.112787,120.668710,1,1,0,3,2,2,0,1520,2,1,2.0,25.130634,121.450528,3.8,ChIJdejlPMWvQjQRIeJNB7fTWvU,14.0,25.132531,121.452336,4.7,ChIJtz-iwkSvQjQRUIZOFQSMM0M,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,南區信義南街６９號六樓之一,104/07/01,550.0,15.5,35.58,60.59%,19.0,1,NaN,Taichung,24.127637,120.673423,3,1,0,3,2,2,0,1839,2,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,12,八里區龍米路一段１３３號四樓,104/07/01,400.0,10.2,39.20,92.17%,28.0,0,NaN,New Taipei,25.126744,121.452093,2,1,0,3,2,2,2,2395,2,0,1.0,25.115406,121.655225,NaN,ChIJJ1IfGJRSXTQRraFxsbBUMfU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,16,汐止區康寧街９１號３樓之１,104/07/01,760.0,31.1,24.46,69.73%,24.0,1,NaN,New Taipei,25.066562,121.620222,1,1,0,2,1,2,8,1086,2,0,8.0,24.984503,121.506902,4.1,ChIJz4XYXW4CaDQRF3YDLsO9XBo,15.0,24.984592,121.506835,NaN,ChIJexCo1EgDaDQR_SHAX4c28UM,NaN,24.981642,121.514525,3.6,ChIJcZARMhMCaDQRzOhBP59D188,108.0,24.981861,121.502577,4.2,ChIJFZFnLN4DaDQR3Uei5TgIfZQ,6.0,24.972753,121.508588,4.1,ChIJf4CTYUsDaDQRMrHAgRnrVGw,68.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,17,汐止區大同路二段３１２巷２號６樓,104/07/01,1000.0,31.6,31.62,82.02%,20.0,1,NaN,New Taipei,25.065980,121.655905,1,1,0,3,2,2,0,1837,2,0,6.0,24.990212,121.232307,4.2,ChIJc4gNDJEhaDQR4heJMCXUrYg,17.0,24.990471,121.231857,NaN,ChIJM7aNi7EhaDQRP1p492EpMS8,NaN,24.993783,121.228872,NaN,ChIJ3e9XdnwhaDQR-QDbX6MtFQQ,NaN,24.993781,121.227066,NaN,ChIJgWW07jshaDQRHHtvJ9ywGck,NaN,24.999332,121.232940,4.2,ChIJF7lJ0MwhaDQRZbt4rPB3piE,96.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,

In [34]:
# Encoding the string dataframe (Regionname), (Council) and (Suburb) columns
ordinal_enc = OrdinalEncoder()

#total_data['Encoded_type'] = ordinal_enc.fit_transform(total_data[['type']]).astype(int)
Encoded_type = total_data[["Encoded_type"]].drop_duplicates().sort_values(by='Encoded_type')

# total_data['Encoded_floor'] = ordinal_enc.fit_transform(total_data[['Floor type/floor height']]).astype(int)
Encoded_floor = total_data[["Encoded_floor"]].drop_duplicates().sort_values(by='Encoded_floor')

# total_data['Encoded_purpose'] = ordinal_enc.fit_transform(total_data[['The main purpose']]).astype(int)
Encoded_purpose = total_data[["Encoded_purpose"]].drop_duplicates().sort_values(by='Encoded_purpose').reset_index().drop(columns="index")
total_data = total_data.loc[total_data["Encoded_purpose"]>=0]
total_data = total_data.loc[total_data["Encoded_type"]>=0]
total_data = total_data.loc[total_data["Encoded_floor"]>=0]
total_data = total_data.reset_index().drop(columns="index")

#Encoded_floor = Encoded_floor.dropna(subset=["Floor type/floor height"]).reset_index().drop(columns="index")
#Encoded_purpose = Encoded_purpose.dropna(subset=["The main purpose"]).reset_index().drop(columns="index")
# Encoded_type.to_csv('../taiwan_dataset/v0/nodes_type_data.csv')
# Encoded_floor.to_csv('../taiwan_dataset/v0/nodes_floor_data.csv')
# Encoded_purpose.to_csv('../taiwan_dataset/v0/nodes_purpose_data.csv')


type_data = total_data["Encoded_type"].copy()
floor_data = total_data["Encoded_floor"].copy()
purpose_data = total_data["Encoded_purpose"].copy()

type_data = type_data.reset_index().rename(columns={"index":"node_id"})
floor_data = floor_data.reset_index().rename(columns={"index":"node_id"})
purpose_data = purpose_data.reset_index().rename(columns={"index":"node_id"})

# print(len(purpose_data))
# type_data = type_data[type_data["Encoded_type"]>=0]
# floor_data = floor_data[floor_data["Encoded_floor"]>=0]
# purpose_data = purpose_data[purpose_data["Encoded_purpose"]>=0]

print(len(purpose_data))

# type_data.to_csv('../taiwan_dataset/v0/edges_type_data.csv')
# floor_data.to_csv('../taiwan_dataset/v0/edges_floor_data.csv')
# purpose_data.to_csv('../taiwan_dataset/v0/edges_purpose_data.csv')

total_data['Encoded_city'] = ordinal_enc.fit_transform(total_data[['City']]).astype(int)
Encoded_city = total_data[["City", "Encoded_city"]].drop_duplicates().sort_values(by='Encoded_city').reset_index().drop(columns="index")
# Encoded_city["Encoded_city"].to_csv('../taiwan_dataset/v0/nodes_city_data.csv')
city_data = total_data["Encoded_city"].copy()
city_data = city_data.reset_index().rename(columns={"index":"node_id"})
city_data = city_data[city_data["Encoded_city"]>=0]

# city_data.to_csv('../taiwan_dataset/v0/edges_city_data.csv')

# total_data_removed =total_data.drop(columns=["type","Floor type/floor height","The main purpose", "Number of transactions", "The current layout of the building", "Remark"])
print(len(total_data))


368345
368345


In [35]:

df_concated=pd.concat([total_data[["hospitals_nearby_0_place_id"]].rename(columns={"hospitals_nearby_0_place_id":"a"}),total_data[["hospitals_nearby_1_place_id"]].rename(columns={"hospitals_nearby_1_place_id":"a"}),total_data[["hospitals_nearby_2_place_id"]].rename(columns={"hospitals_nearby_2_place_id":"a"}),total_data[["hospitals_nearby_3_place_id"]].rename(columns={"hospitals_nearby_3_place_id":"a"}),total_data[["hospitals_nearby_4_place_id"]].rename(columns={"hospitals_nearby_4_place_id":"a"})],ignore_index=True)
ordinal_enc = OrdinalEncoder()
res = ordinal_enc.fit_transform(df_concated)
res = pd.DataFrame(res.astype(int))
len_res = int(len(res)/5)
res

,0
0,395
1,327
2,2397
3,171
4,1762
...,...
1841720,2473
1841721,1033
1841722,663
1841723,853


In [36]:
total_data["hospitals_0_unique"] = res[:len_res]
total_data["hospitals_1_unique"] = res[len_res:len_res*2].reset_index().drop("index",axis=1)
total_data["hospitals_2_unique"] = res[len_res*2:len_res*3].reset_index().drop("index",axis=1)
total_data["hospitals_3_unique"] = res[len_res*3:len_res*4].reset_index().drop("index",axis=1)
total_data["hospitals_4_unique"] = res[len_res*4:len_res*5].reset_index().drop("index",axis=1)
Encoded_hospital_0 = total_data[["hospitals_nearby_0_geometry_lat","hospitals_nearby_0_geometry_lng","hospitals_nearby_0_rating", "hospitals_nearby_0_user_ratings_total", "hospitals_0_unique"]].drop_duplicates().sort_values(by='hospitals_0_unique')
Encoded_hospital_1 = total_data[["hospitals_nearby_1_geometry_lat","hospitals_nearby_1_geometry_lng","hospitals_nearby_1_rating", "hospitals_nearby_1_user_ratings_total", "hospitals_1_unique"]].drop_duplicates().sort_values(by='hospitals_1_unique')
Encoded_hospital_2 = total_data[["hospitals_nearby_2_geometry_lat","hospitals_nearby_2_geometry_lng","hospitals_nearby_2_rating", "hospitals_nearby_2_user_ratings_total", "hospitals_2_unique"]].drop_duplicates().sort_values(by='hospitals_2_unique')
Encoded_hospital_3 = total_data[["hospitals_nearby_3_geometry_lat","hospitals_nearby_3_geometry_lng","hospitals_nearby_3_rating", "hospitals_nearby_3_user_ratings_total", "hospitals_3_unique"]].drop_duplicates().sort_values(by='hospitals_3_unique')
Encoded_hospital_4 = total_data[["hospitals_nearby_4_geometry_lat","hospitals_nearby_4_geometry_lng","hospitals_nearby_4_rating", "hospitals_nearby_4_user_ratings_total", "hospitals_4_unique"]].drop_duplicates().sort_values(by='hospitals_4_unique')

In [37]:
Encoded_hospital_2

,hospitals_nearby_2_geometry_lat,hospitals_nearby_2_geometry_lng,hospitals_nearby_2_rating,hospitals_nearby_2_user_ratings_total,hospitals_2_unique
6,NaN,NaN,NaN,NaN,-9223372036854775808
166545,22.763871,121.146279,5.0,1.0,1
28576,24.182317,120.617897,3.4,201.0,3
76,25.011923,121.518050,3.2,5.0,4
2979,24.966109,121.221754,2.9,11.0,5
...,...,...,...,...,...
5013,24.983509,121.446292,3.7,29.0,2633
221,24.159088,120.689163,4.5,15.0,2635
2359,25.066133,121.537792,4.2,157.0,2636
164485,23.018201,120.243733,5.0,5.0,2638


In [38]:
Encoded_hospital_0.drop(index=5,inplace=True)
Encoded_hospital_1.drop(index=5,inplace=True)
Encoded_hospital_2.drop(index=5,inplace=True)
Encoded_hospital_3.drop(index=2,inplace=True)
Encoded_hospital_4.drop(index=2,inplace=True)

In [39]:
# Filling the missing rating values with 0
Encoded_hospital_0["hospitals_nearby_0_rating"].fillna((0), inplace=True)
Encoded_hospital_0["hospitals_nearby_0_user_ratings_total"].fillna((0), inplace=True)
Encoded_hospital_1["hospitals_nearby_1_rating"].fillna((0), inplace=True)
Encoded_hospital_1["hospitals_nearby_1_user_ratings_total"].fillna((0), inplace=True)
Encoded_hospital_2["hospitals_nearby_2_rating"].fillna((0), inplace=True)
Encoded_hospital_2["hospitals_nearby_2_user_ratings_total"].fillna((0), inplace=True)
Encoded_hospital_3["hospitals_nearby_3_rating"].fillna((0), inplace=True)
Encoded_hospital_3["hospitals_nearby_3_user_ratings_total"].fillna((0), inplace=True)
Encoded_hospital_4["hospitals_nearby_4_rating"].fillna((0), inplace=True)
Encoded_hospital_4["hospitals_nearby_4_user_ratings_total"].fillna((0), inplace=True)
save0_train = Encoded_hospital_0.rename(columns={"hospitals_nearby_0_place_id":"a","hospitals_nearby_0_geometry_lng":"b","hospitals_nearby_0_rating":"c", "hospitals_nearby_0_user_ratings_total":"d", "hospitals_0_unique":"e","hospitals_nearby_0_geometry_lat":"f"}).copy()
save1_train = Encoded_hospital_1.rename(columns={"hospitals_nearby_1_place_id":"a","hospitals_nearby_1_geometry_lng":"b","hospitals_nearby_1_rating":"c", "hospitals_nearby_1_user_ratings_total":"d", "hospitals_1_unique":"e","hospitals_nearby_1_geometry_lat":"f"}).copy()
save2_train = Encoded_hospital_2.rename(columns={"hospitals_nearby_2_place_id":"a","hospitals_nearby_2_geometry_lng":"b","hospitals_nearby_2_rating":"c", "hospitals_nearby_2_user_ratings_total":"d", "hospitals_2_unique":"e","hospitals_nearby_2_geometry_lat":"f"}).copy()
save3_train = Encoded_hospital_3.rename(columns={"hospitals_nearby_3_place_id":"a","hospitals_nearby_3_geometry_lng":"b","hospitals_nearby_3_rating":"c", "hospitals_nearby_3_user_ratings_total":"d", "hospitals_3_unique":"e","hospitals_nearby_3_geometry_lat":"f"}).copy()
save4_train = Encoded_hospital_4.rename(columns={"hospitals_nearby_4_place_id":"a","hospitals_nearby_4_geometry_lng":"b","hospitals_nearby_4_rating":"c", "hospitals_nearby_4_user_ratings_total":"d", "hospitals_4_unique":"e","hospitals_nearby_4_geometry_lat":"f"}).copy()
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).drop_duplicates(subset='e').sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,NaN,NaN,0.0,0.0,-9223372036854775808
5418,24.984895,121.307601,0.0,0.0,0
2295,22.763871,121.146279,5.0,1.0,1
1,24.113320,120.601844,5.0,1.0,2
2,24.182317,120.617897,3.4,201.0,3
...,...,...,...,...,...
2293,25.066133,121.537792,4.2,157.0,2636
7119,23.159255,120.176705,0.0,0.0,2637
3797,23.018201,120.243733,5.0,5.0,2638
5416,23.229879,120.182534,3.6,36.0,2639


In [40]:
scaler_lat = MinMaxScaler(feature_range = (1,10))
scaler_lat.fit(train["geometry_lat"].values.reshape(-1, 1))
scaler_lng = MinMaxScaler(feature_range = (1,10))
scaler_lng.fit(train["geometry_lng"].values.reshape(-1, 1))
scaler_0 = MinMaxScaler(feature_range = (1,10))
saveall_df_train['f'] = scaler_lat.transform(saveall_df_train['f'].values.reshape(-1, 1))
saveall_df_train['b'] = scaler_lng.transform(saveall_df_train['b'].values.reshape(-1, 1))
saveall_df_train['d'] = scaler_0.fit_transform(saveall_df_train['d'].values.reshape(-1, 1))
# saveall_df_train['01'] = 0
# saveall_df_train['02'] = 0
# saveall_df_train['03'] = 0
# saveall_df_train['04'] = 0
# saveall_df_train['05'] = 0
# saveall_df_train['06'] = 0
# saveall_df_train['07'] = 0
# saveall_df_train['08'] = 0
# saveall_df_train['09'] = 0
# saveall_df_train['10'] = 0
# saveall_df_train['11'] = 0
# saveall_df_train['12'] = 0
# saveall_df_train['13'] = 0
# saveall_df_train['14'] = 0
# saveall_df_train['15'] = 0
# saveall_df_train['16'] = 0
# saveall_df_train['17'] = 0
# saveall_df_train['18'] = 0

saveall_df_train

,f,b,c,d,e
0,NaN,NaN,0.0,1.000000,-9223372036854775808
5418,1.790559,8.762010,0.0,1.000000,0
2295,1.201773,8.751952,5.0,1.000695,1
1,1.559508,8.718010,5.0,1.000695,2
2,1.577799,8.719011,3.4,1.139777,3
...,...,...,...,...,...
2293,1.812095,8.776361,4.2,1.109179,2636
7119,1.306588,8.691505,0.0,1.000000,2637
3797,1.269195,8.695684,5.0,1.003477,2638
5416,1.325310,8.691868,3.6,1.025035,2639


In [41]:
saveall_df_train['feat'] = saveall_df_train.drop("e",axis=1).astype(str).agg(', '.join, axis=1)
saveall_df_train = saveall_df_train[["e", "feat"]]
saveall_df_train

,e,feat
0,-9223372036854775808,"nan, nan, 0.0, 1.0"
5418,0,"1.7905591655479665, 8.762009644719349, 0.0, 1.0"
2295,1,"1.2017731446508781, 8.751952152773145, 5.0, 1...."
1,2,"1.5595075314345612, 8.718009873220135, 5.0, 1...."
2,3,"1.5777985615486614, 8.719010670423708, 3.4, 1...."
...,...,...
2293,2636,"1.8120951107621819, 8.776360711780978, 4.2, 1...."
7119,2637,"1.3065883162431886, 8.691504950554362, 0.0, 1.0"
3797,2638,"1.269195192925343, 8.695683775138235, 5.0, 1.0..."
5416,2639,"1.3253102609028744, 8.691868385425428, 3.6, 1...."


In [42]:
saveall_df_train.to_csv("nodes_hospitals_data.csv")

In [43]:
hospital_data_0 = total_data["hospitals_0_unique"].copy()
hospital_data_1 = total_data["hospitals_1_unique"].copy()
hospital_data_2 = total_data["hospitals_2_unique"].copy()
hospital_data_3 = total_data["hospitals_3_unique"].copy()
hospital_data_4 = total_data["hospitals_4_unique"].copy()

hospital_data_0 = hospital_data_0.reset_index().rename(columns={"index":"node_id"})
hospital_data_1 = hospital_data_1.reset_index().rename(columns={"index":"node_id"})
hospital_data_2 = hospital_data_2.reset_index().rename(columns={"index":"node_id"})
hospital_data_3 = hospital_data_3.reset_index().rename(columns={"index":"node_id"})
hospital_data_4 = hospital_data_4.reset_index().rename(columns={"index":"node_id"})

hospital_data_0 = hospital_data_0[hospital_data_0["hospitals_0_unique"]>=0]
hospital_data_1 = hospital_data_1[hospital_data_1["hospitals_1_unique"]>=0]
hospital_data_2 = hospital_data_2[hospital_data_2["hospitals_2_unique"]>=0]
hospital_data_3 = hospital_data_3[hospital_data_3["hospitals_3_unique"]>=0]
hospital_data_4 = hospital_data_4[hospital_data_4["hospitals_4_unique"]>=0]

hospital_data_0 = hospital_data_0.rename(columns={"hospitals_0_unique":"dest_id"}).copy()
hospital_data_1 = hospital_data_1.rename(columns={"hospitals_1_unique":"dest_id"}).copy()
hospital_data_2 = hospital_data_2.rename(columns={"hospitals_2_unique":"dest_id"}).copy()
hospital_data_3 = hospital_data_3.rename(columns={"hospitals_3_unique":"dest_id"}).copy()
hospital_data_4 = hospital_data_4.rename(columns={"hospitals_4_unique":"dest_id"}).copy()

saveall_df = pd.concat([hospital_data_0,hospital_data_1,hospital_data_2,hospital_data_3,hospital_data_4], ignore_index=True)
saveall_df

,node_id,dest_id
0,0,395
1,1,327
2,2,2397
3,3,171
4,4,1762
...,...,...
1491048,368340,2473
1491049,368341,1033
1491050,368342,663
1491051,368343,853


In [44]:
hospital_data_0

,node_id,dest_id
0,0,395
1,1,327
2,2,2397
3,3,171
4,4,1762
...,...,...
368340,368340,173
368341,368341,2091
368342,368342,207
368343,368343,703


In [45]:
saveall_df.to_csv('edges_hospital_data.csv')
